### Contents:
- [1. Import Libraries](#1.-Import-Libraries)
- [2. Data Import](#2.-Data-Import)
- [3. Data Cleaning](#3.-Data-Cleaning)
    - [3.1. Data](#3.1.-Data)
    - [3.2. Missing Values](#3.2.-Missing-Values)
    - [3.3. Merged Rows](#3.3.-Merged-Rows)
    - [3.4. Features Transformation](#3.4.-Features-Transformation)

## 1. Import Libraries
---

In [1]:
import pandas as pd
import numpy as np
import datetime

## 2. Data Import
---

In [2]:
train = pd.read_csv("../Datasets/train.csv")
test = pd.read_csv("../Datasets/test.csv")

In [3]:
# display top 5 rows of train dataset
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


## 3. Data Cleaning
---

## 3.1. Data

In [4]:
# train set dimensions
train.shape

(10506, 12)

In [5]:
# check the data types - train set
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    10506 non-null  object 
 1   Address                 10506 non-null  object 
 2   Species                 10506 non-null  object 
 3   Block                   10506 non-null  int64  
 4   Street                  10506 non-null  object 
 5   Trap                    10506 non-null  object 
 6   AddressNumberAndStreet  10506 non-null  object 
 7   Latitude                10506 non-null  float64
 8   Longitude               10506 non-null  float64
 9   AddressAccuracy         10506 non-null  int64  
 10  NumMosquitos            10506 non-null  int64  
 11  WnvPresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


## 3.2. Missing Values

In [6]:
train.isnull().sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

## 3.3. Merge Rows

In [7]:
# group by date, location, trap, and other variables and sum up the number of mosquitos
train_clean = pd.DataFrame(train.groupby(["Date",
                                          "Address", 
                                          "Block", 
                                          "Street", 
                                          "AddressNumberAndStreet",
                                          "Latitude", 
                                          "Longitude", 
                                          "Species",
                                          "Trap",
                                          "AddressAccuracy", 
                                          "WnvPresent"
                                         ],
                                         as_index=True).\
                            agg(tot_mosquitos=("NumMosquitos", "sum"))).\
                            reset_index()

# display the top 5 rows of the dataset
train_clean.head()

,Date,Address,Block,Street,AddressNumberAndStreet,Latitude,Longitude,Species,Trap,AddressAccuracy,WnvPresent,tot_mosquitos
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",11,W ROOSEVELT,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,CULEX PIPIENS/RESTUANS,T048,8,0,1
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",11,W ROOSEVELT,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,CULEX RESTUANS,T048,8,0,2
2,2007-05-29,"1100 South Peoria Street, Chicago, IL 60608, USA",11,S PEORIA ST,"1100 S PEORIA ST, Chicago, IL",41.862292,-87.648860,CULEX RESTUANS,T091,8,0,1
3,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",11,W CHICAGO,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,CULEX RESTUANS,T049,8,0,1
4,2007-05-29,"1500 North Long Avenue, Chicago, IL 60651, USA",15,N LONG AVE,"1500 N LONG AVE, Chicago, IL",41.907645,-87.760886,CULEX RESTUANS,T153,8,0,1


## 3.4. Features Transformation

In [8]:
# identify the list of traps that are in both train and test set
train_trap = list(train["Trap"].unique())
test_trap = list(test["Trap"].unique())

# identify the list of traps that are in the test set but are not in the train set
trap_gap = list(set(test_trap) - set(train_trap))
print(trap_gap)

['T065A', 'T218C', 'T128A', 'T090B', 'T218B', 'T090C', 'T234', 'T090A', 'T002B', 'T002A', 'T200B', 'T218A', 'T200A']


In [9]:
# update train_trap to include traps that are missing
train_trap.extend(trap_gap)

In [10]:
# transformation  - create a function to transform features
def transform_catfeatures(dataframe):
    # species
    dataframe["pipiens/restuans"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX PIPIENS/RESTUANS" else 0)
    dataframe["restuans"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX RESTUANS" else 0)
    dataframe["pipiens"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX PIPIENS" else 0)
    dataframe["salinarius"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX SALINARIUS" else 0)
    dataframe["territans"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX TERRITANS" else 0)
    dataframe["tarsalis"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX TARSALIS" else 0)
    dataframe["erraticus"] = dataframe["Species"].map(lambda x: 1 if x == "CULEX ERRATICUS" else 0)
    dataframe["unspecified"] = dataframe["Species"].map(lambda x: 1 if x == "UNSPECIFIED CULEX" else 0)
    
    # date
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    dataframe["month_year"] = pd.to_datetime(dataframe["Date"]).dt.to_period("M")
    dataframe["month_year"] = dataframe["month_year"].astype("str")
    dataframe["year"] = pd.DatetimeIndex(dataframe["Date"]).year
    dataframe["month"] = pd.DatetimeIndex(dataframe["Date"]).month
    dataframe["day"] = pd.DatetimeIndex(dataframe["Date"]).day
    
    # month
    months_list = list(dataframe["month"].unique())
    for i, month in enumerate(months_list):
        dataframe[str(month)] = dataframe["month"].map(lambda x: 1 if x == month else 0) 
     
    # trap id
    for i, trap in enumerate(train_trap):
        dataframe[trap] = dataframe["Trap"].map(lambda x: 1 if x == trap else 0)
     
    return dataframe

In [11]:
# new train dataset after cleaning and features transformation
train_clean = transform_catfeatures(train_clean)

# drop 5 from the new train dataset as the month May is not in the test dataset
train_clean = train_clean.drop(columns=["5"])

In [12]:
# new test dataset after cleaning and features transformation
test_clean = transform_catfeatures(test)

In [13]:
# new train dataset dimensions
train_clean.shape

(8610, 178)

In [14]:
# save the new datasets to csv
train_clean.to_csv("../Datasets/train_clean.csv", index=False)
test_clean.to_csv("../Datasets/test_clean.csv", index=False)